In [18]:
import pandas as pd
import httplib2
from pathlib import Path
from bs4 import BeautifulSoup, SoupStrainer
pd.set_option('display.max_rows', 200)

In [19]:
def get_data():
    """
    The link to the Sponsor list  is updated. This function is designed to fetch the newest version
    """
    # Get webpage
    http = httplib2.Http()
    status, response = http.request('https://www.gov.uk/government/publications/register-of-licensed-sponsors-workers')
    
    # Search for the link
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        if link.has_attr('href') and link['href'].startswith("https://assets.publishing.service.gov.uk/"):
            file_name = link['href'].split('/')[-1]
            print("\nLast updated:", file_name[:10])
            
            # Fetch the latest data, cache and use old data if there is no update
            if Path(file_name).is_file():
                df = pd.read_csv(file_name)
            else:
                df = pd.read_csv(link['href'])
                df.to_csv(file_name)
                
    return df

In [24]:
# Fetch data
data = get_data()


Last updated: 2022-09-08


In [21]:
def search(word, based_on="org", route="Skilled Worker"):
    """
    based_on: Use "org", "toc" or "c" to search based on "Organisation Name", "Town/City" or "County" repectively. 
    word: This is the text to be searched
    route: Any of the elements present in data['Route'].unique(). default is "Skilled Worker".
    
    returns: A dataframe showing the results of the search.
    """
    
    expand_arg = {'org': "Organisation Name", 'toc': "Town/City", 'c': "County"}
    column = expand_arg[based_on]
    df = data.dropna(subset=[column])
    sponsor = df[df[column].str.contains(word, case=False)]
    return sponsor[sponsor["Route"] == route]

In [23]:
search("Quantip")

,Unnamed: 0,Organisation Name,Town/City,County,Type & Rating,Route
43355,43355,Quantiphi Limited,London,NaN,Worker (A rating),Skilled Worker
